# Attachment Protocol - Bob
## Role: Receiver

## This notebook should be run alongside [Alice](http://localhost:8888/notebooks/attachment.ipynb), who will send Bob two attachments.

In this notebook Bob instatiates his controller and sets up a handler for attachment protocol messages, all files he receives he will save in the [received_files folder](http://localhost:8889/tree/received_files). It should currently be empty.


###  Intialise Controller

In [1]:
%autoawait
import time
import asyncio
import base64
from attachment_controller.attachment_controller import AttachmentController
    
WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 8052
WEBHOOK_BASE = ""
ADMIN_URL = "http://bob-agent:8051"

# Based on the aca-py agent you wish to control
attach_controller = AttachmentController(webhook_host=WEBHOOK_HOST, webhook_port=WEBHOOK_PORT,
                                       webhook_base=WEBHOOK_BASE, admin_url=ADMIN_URL)

IPython autoawait is `on`, and set to use `asyncio`


###  Check if active connection exists

In [2]:
response = await attach_controller.connections.get_connections()
results = response['results']
print("Results : ", results)
if len(results) > 0:
    connection = response['results'][0]
    print("Connection :", connection)
    if connection['state'] == 'active':       
        connection_id = connection["connection_id"]
        print("Active Connection ID : ", connection_id)
    else:
        print("Connection is still progressing to active state, retry in a few moments")
else:
    print("You must create a connection")


Results :  [{'connection_id': 'cc549960-d6ba-4f5e-9605-97f5d1412404', 'their_did': 'WH7bh7MxhMdUgiDY4wT4yM', 'accept': 'manual', 'updated_at': '2020-11-03 19:13:55.246145Z', 'state': 'active', 'invitation_key': 'EptJRdmoA7P7TU8qo91Fykim1YUPrWpsZomAuihX88pc', 'their_label': 'Alice', 'my_did': 'XsKkSFopp73qWzUUChNQvw', 'initiator': 'self', 'routing_state': 'none', 'created_at': '2020-11-03 19:13:39.679948Z', 'invitation_mode': 'once'}]
Connection : {'connection_id': 'cc549960-d6ba-4f5e-9605-97f5d1412404', 'their_did': 'WH7bh7MxhMdUgiDY4wT4yM', 'accept': 'manual', 'updated_at': '2020-11-03 19:13:55.246145Z', 'state': 'active', 'invitation_key': 'EptJRdmoA7P7TU8qo91Fykim1YUPrWpsZomAuihX88pc', 'their_label': 'Alice', 'my_did': 'XsKkSFopp73qWzUUChNQvw', 'initiator': 'self', 'routing_state': 'none', 'created_at': '2020-11-03 19:13:39.679948Z', 'invitation_mode': 'once'}
Active Connection ID :  cc549960-d6ba-4f5e-9605-97f5d1412404


### Instantiate Listener

The "attachment" event is emitted using PyPubSub when the controller receives a atatchment webhook from the agent. This happens everytime the agent receives an attachment.

In this handler we receive the attachment payload, decode it and save it in the received files folder using the filename provided in the protocol. 

Currently, this has been tested for basic text files and for images. Feel free to try additional file types.

In [3]:

def attach_handler(payload):
    connection_id = payload['connection_id']
    print("Handle Attachment", payload, connection_id)
    print("Attachment Received")
    data = payload['content']
    filename = payload['filename']
    file_content = base64.b64decode(data)
#     fh = open("image_received.png", "wb")
#     fh.write(text)
#     fh.close()
    filepath = f"received_files/{filename}"
    f = open(filepath,'wb')
    f.write(file_content)
    f.close()
attach_listener = {
    "handler": attach_handler,
    "topic": "attachment"
}
loop = asyncio.get_event_loop()
loop.create_task(attach_controller.listen_webhooks())
attach_controller.register_listeners([attach_listener], defaults=True)

Handle Attachment {'connection_id': 'cc549960-d6ba-4f5e-9605-97f5d1412404', 'message_id': '4a1e2d88-7b8f-4eaa-b272-93d6598a7e74', 'attachment_message': 'Here is the data you wanted', 'content': 'VGhpcyBpcyBhIHRlc3QgZmlsZSB0byBhdHRhY2gu', 'filename': 'test_file.txt', 'state': 'received'} cc549960-d6ba-4f5e-9605-97f5d1412404
Attachment Received
Handle Attachment {'connection_id': 'cc549960-d6ba-4f5e-9605-97f5d1412404', 'message_id': 'b5540c5d-7df7-4c55-9c57-59fa64296da6', 'attachment_message': 'Here is the data you wanted', 'content': '...', 'filename': 'openmined.jpg', 'state': 'received'} cc549960-d6ba-4f5e-9605-97f5d1412404
Attachment Received



# End of Tutorial

Be sure to terminate the controller so you can run another tutorial.


In [11]:
response = await attach_controller.terminate()
print(response)


None
